# Анализ надежности клиентов

## Описание и цель проекта

Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок.
 
  Результаты исследования будут учтены при построении модели кредитного скоринга — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.
  В конце исследования нужно ответить на следующие вопросы:
  1. Есть ли зависимость между количеством детей и возвратом кредита в срок?
  2. Есть ли зависимость между семейным положением и возвратом кредита в срок?
  3. Есть ли зависимость между уровнем дохода и возвратом кредита в срок?
  4. Как разные цели кредита влияют на его возврат в срок?
  
  **Ход исследования**

Данные о поведении пользователей получим из файла '/datasets/data.csv'. О качестве данных ничего не известно. Поэтому перед проверкой гипотез понадобится обзор данных. 



## Входные данные

Cтатистика о платёжеспособности клиентов


## Описание данных

children — количество детей в семье

days_employed — общий трудовой стаж в днях

dob_years — возраст клиента в годах

education — уровень образования клиента

education_id — идентификатор уровня образования

family_status — семейное положение

family_status_id — идентификатор семейного положения

gender — пол клиента

income_type — тип занятости

debt — имел ли задолженность по возврату кредитов


total_income — ежемесячный доход

purpose — цель получения кредита

## Шаги проекта

 1. Обзор данных.
 2. Предобработка данных.
 3. Ответы на вопросы.
 4. Общий вывод

## 1 Обзор данных

1.1 Импортируем библиотеку pandas. Считаем данные из csv-файла в датафрейм и сохраним в переменную `data`. 



In [1]:
# Чтобы избавиться от вывода предупреждений, нужно добавить эти две строчки
import warnings
warnings.simplefilter("ignore")

In [2]:
import pandas as pd

try:
    data = pd.read_csv('/datasets/data.csv')
except:
    data = pd.read_csv('https://code.s3.yandex.net/datasets/data.csv')

In [5]:
# Выведим первые 20 строчек датафрейма data на экран
data.head(5)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


Выведим основную информацию о датафрейме с помощью метода info().

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Названия столбцов соотвествуют принятому стилю. Имеются пропуски в колонках days_employed и total_income. На сследующем этапе проработаем данные и устраним пропуски если это возможно.

## 2 Предобработка данных

Обработаем пропуски и проверим данные на корректность.

### 2.1 Удаление пропусков

Выведим количество пропущенных значений для каждого столбца.

In [5]:
data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

 В двух столбцах есть пропущенные значения. Один из них — days_employed. Пропуски в этом столбце мы обработаем на следующем этапе. Другой столбец с пропущенными значениями — total_income — хранит данные о доходах. На сумму дохода сильнее всего влияет тип занятости, поэтому заполнить пропуски в этом столбце нужно медианным значением по каждому типу из столбца income_type. Например, у человека с типом занятости сотрудник пропуск в столбце total_income должен быть заполнен медианным доходом среди всех записей с тем же типом.

In [6]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['total_income'].isna()), 'total_income'] = \
    data.loc[(data['income_type'] == t), 'total_income'].median()


### 2.2 Обработка аномальных значений

В данных могут встречаться артефакты (аномалии) — значения, которые не отражают действительность и появились по какой-то ошибке. Таким артефактом будет отрицательное количество дней трудового стажа в столбце days_employed. Для реальных данных это нормально. Обработаем значения в этом столбце: замените все отрицательные значения положительными с помощью метода abs().

In [7]:
data['days_employed'] = data['days_employed'].abs()

Для каждого типа занятости выведите медианное значение трудового стажа days_employed в днях.

In [8]:
data.groupby('income_type')['days_employed'].agg('median')

income_type
безработный        366413.652744
в декрете            3296.759962
госслужащий          2689.368353
компаньон            1547.382223
пенсионер          365213.306266
предприниматель       520.848083
сотрудник            1574.202821
студент               578.751554
Name: days_employed, dtype: float64

У двух типов (безработные и пенсионеры) получаются аномально большие значения. Исправить такие значения сложно, поэтому оставим их как есть. Тем более этот столбец не понадобится нам для исследования.

Выведим перечень уникальных значений столбца children.

In [9]:
data['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5], dtype=int64)

 В столбце children есть два аномальных значения. Удалим строки, в которых встречаются такие аномальные значения из датафрейма data.

In [10]:
data = data[(data['children'] != -1) & (data['children'] != 20)]

In [11]:
# Ещё раз выведите перечень уникальных значений столбца children, 
# чтобы убедиться, что артефакты удалены.
data['children'].unique()

array([1, 0, 3, 2, 4, 5], dtype=int64)

### 2.3 Удаление пропусков (продолжение)

Заполните пропуски в столбце days_employed медианными значениями по каждого типа занятости income_type.

In [12]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['days_employed'].isna()), 'days_employed'] = \
    data.loc[(data['income_type'] == t), 'days_employed'].median()

In [13]:
# Убедитесь, что все пропуски заполнены.
data.isna().sum()


children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

### 2.4 Изменение типов данных

Заменим вещественный тип данных в столбце total_income на целочисленный с помощью метода astype().

In [14]:
data['total_income'] = data['total_income'].astype('int64')

### 2.5 Обработка дубликатов

Обработаем неявные дубликаты в столбце education.

In [15]:
data['education'].unique()

array(['высшее', 'среднее', 'Среднее', 'СРЕДНЕЕ', 'ВЫСШЕЕ',
       'неоконченное высшее', 'начальное', 'Высшее',
       'НЕОКОНЧЕННОЕ ВЫСШЕЕ', 'Неоконченное высшее', 'НАЧАЛЬНОЕ',
       'Начальное', 'Ученая степень', 'УЧЕНАЯ СТЕПЕНЬ', 'ученая степень'],
      dtype=object)

 В этом столбце есть одни и те же значения, но записанные по-разному: с использованием заглавных и строчных букв. Приведем их к нижнему регистру. Проверим остальные столбцы.

In [16]:
data['education'] = data['education'].str.lower()

In [17]:
data.duplicated().sum()

71

In [18]:
data = data.drop_duplicates()

### 2.6 Категоризация данных

 На основании диапазонов, указанных ниже, создадим в датафрейме data столбец total_income_category с категориями:

0–30000 — 'E';

30001–50000 — 'D';

50001–200000 — 'C';

200001–1000000 — 'B';

1000001 и выше — 'A'.

Например, кредитополучателю с доходом 25000 нужно назначить категорию 'E', а клиенту, получающему 235000, — 'B'. 

In [19]:
def categorize_income(income):
    try:
        if 0 <= income <= 30000:
            return 'E'
        elif 30001 <= income <= 50000:
            return 'D'
        elif 50001 <= income <= 200000:
            return 'C'
        elif 200001 <= income <= 1000000:
            return 'B'
        elif income >= 1000001:
            return 'A'
    except:
        pass

In [26]:
data['total_income_category'] = data['total_income'].apply(categorize_income)
data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category,purpose_category
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,B,операции с недвижимостью
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,C,операции с автомобилем
2,0,5623.422610,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,C,операции с недвижимостью
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,B,получение образования
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,C,проведение свадьбы
5,0,926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья,B,операции с недвижимостью
6,0,2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем,B,операции с недвижимостью
7,0,152.779569,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование,C,получение образования
8,2,6929.865299,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы,C,проведение свадьбы
9,0,2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи,C,операции с недвижимостью


Выведим на экран перечень уникальных целей взятия кредита из столбца purpose.

In [21]:
data['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

Создадим функцию, которая на основании данных из столбца purpose сформирует новый столбец purpose_category, в который войдут следующие категории:

* 'операции с автомобилем',

* 'операции с недвижимостью',

* 'проведение свадьбы',

* 'получение образования'.

Например, если в столбце purpose находится подстрока 'на покупку автомобиля', то в столбце purpose_category должна появиться строка 'операции с автомобилем'.


In [22]:
def categorize_purpose(row):
    try:
        if 'автом' in row:
            return 'операции с автомобилем'
        elif 'жил' in row or 'недвиж' in row:
            return 'операции с недвижимостью'
        elif 'свад' in row:
            return 'проведение свадьбы'
        elif 'образов' in row:
            return 'получение образования'
    except:
        return 'нет категории'

In [25]:
data['purpose_category'] = data['purpose'].apply(categorize_purpose)
data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category,purpose_category
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,B,операции с недвижимостью
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,C,операции с автомобилем
2,0,5623.422610,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,C,операции с недвижимостью
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,B,получение образования
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,C,проведение свадьбы
5,0,926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья,B,операции с недвижимостью
6,0,2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем,B,операции с недвижимостью
7,0,152.779569,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование,C,получение образования
8,2,6929.865299,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы,C,проведение свадьбы
9,0,2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи,C,операции с недвижимостью


## 3. Исследуйте данные и ответьте на вопросы

### 3.1 Есть ли зависимость между количеством детей и возвратом кредита в срок?

Создадим сводную таблицу (data_pivot_children) в которой выведем группы клинтов по количеству детей. Посчитаем количество клиентов в каждой группе, и для каждой группы посчитаем количество клиентов имеющих задолжности по кредитам.

In [27]:
data_pivot_children = data.pivot_table(index=['children'], values=['debt'], aggfunc=('count', 'sum'))
data_pivot_children

debt      
          count   sum
children             
0         14091  1063
1          4808   444
2          2052   194
3           330    27
4            41     4
5             9     0

В таблице создадим колонку debt_children_percent. В ней выведем долю клиентов (в %) c задолжностью от общего количества в каждой группе округленное до одного знака после заяпятой.

In [28]:
data_pivot_children['debt_children_percent'] = data_pivot_children['debt','sum'] / data_pivot_children['debt','count'] * 100
data_pivot_children['debt_children_percent'] = data_pivot_children['debt_children_percent'].round(1)

data_pivot_children

debt       debt_children_percent
          count   sum                      
children                                   
0         14091  1063                   7.5
1          4808   444                   9.2
2          2052   194                   9.5
3           330    27                   8.2
4            41     4                   9.8
5             9     0                   0.0

В группе с 5 детьми слишком мало клиентов чтобы принимать ее во внимание при анализе, поэтрому нужно ее исключить.

In [29]:
data_pivot_children = data_pivot_children.loc[data_pivot_children['debt_children_percent'] != 0]
data_pivot_children

debt       debt_children_percent
          count   sum                      
children                                   
0         14091  1063                   7.5
1          4808   444                   9.2
2          2052   194                   9.5
3           330    27                   8.2
4            41     4                   9.8

**Вывод:** Клиенты без детей отдают кредиты чаще чем клиеты имеющие детей. Если исключить группы с недостаточным количеством клиентов для анализа (это группа клиентов с 5 детьми), можно сделать вывод о том что, чем детей больше, тем вероятность задолжности возрастает.

#### 3.2 Есть ли зависимость между семейным положением и возвратом кредита в срок?

Создадим сводную таблицу (data_pivot_family_status) в которой выведем группы клинтов по их семейному положению. Посчитаем количество клиентов в каждой группе и для каждой группы посчитаем количество клиентов имеющих задолжности по кредитам.

In [30]:
data_pivot_family_status = data.pivot_table(index=['family_status'], values=['debt'], aggfunc=('count', 'sum'))
data_pivot_family_status


debt     
                       count  sum
family_status                    
Не женат / не замужем   2796  273
в разводе               1189   84
вдовец / вдова           951   63
гражданский брак        4134  385
женат / замужем        12261  927

Из таблицы data_pivot_family_status Высчитаем процентное отношение клентов с задолжностью к общему числу клиетнов. Данные выведем в колонке family_status_debt.

In [31]:
data_pivot_family_status['family_status_debt'] = data_pivot_family_status['debt','sum'] / data_pivot_family_status['debt','count'] *100
data_pivot_family_status['family_status_debt'] = data_pivot_family_status['family_status_debt'].round(1)
data_pivot_family_status

debt      family_status_debt
                       count  sum                   
family_status                                       
Не женат / не замужем   2796  273                9.8
в разводе               1189   84                7.1
вдовец / вдова           951   63                6.6
гражданский брак        4134  385                9.3
женат / замужем        12261  927                7.6

**Вывод:** Клиенты не состоящие в официальном браке чаще не возвращают кредиты. Тогда как овдовевшие клиенты делают это реже всех остальных.

### 3.3 Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

Сгруппируем клинтов по столбцу (total_income_category), посчитаем количество клиентов в каждой группе, и клиентов имевших задолжности по кредитам.

In [32]:
data_categ_group_pivot = data.pivot_table(index=['total_income_category'], values=['debt'], aggfunc=('count', 'sum'))
data_categ_group_pivot 

debt      
                       count   sum
total_income_category             
A                         25     2
B                       5014   354
C                      15921  1353
D                        349    21
E                         22     2

Высчитаем процентное отношение клентов с задолжностью к общему числу клиетнов. Данные выведем в колонке categ_income_percent.

In [33]:
data_categ_group_pivot['categ_income_percent'] = data_categ_group_pivot['debt','sum'] / data_categ_group_pivot['debt','count'] *100
data_categ_group_pivot['categ_income_percent'] = data_categ_group_pivot['categ_income_percent'].round(1)
data_categ_group_pivot

debt       categ_income_percent
                       count   sum                     
total_income_category                                  
A                         25     2                  8.0
B                       5014   354                  7.1
C                      15921  1353                  8.5
D                        349    21                  6.0
E                         22     2                  9.1

Исключим из таблицы группы с недостаточным количеством клиентов. оставим только группы в которых количество клиенто больше 5000.

In [34]:
data_categ_group_pivot = data_categ_group_pivot.loc[data_categ_group_pivot['debt','count'] > 5000]
data_categ_group_pivot

debt       categ_income_percent
                       count   sum                     
total_income_category                                  
B                       5014   354                  7.1
C                      15921  1353                  8.5

**Вывод:** Проанализировав данную выборку можно предположить, что клиенты с уровнем дохода от 50000 до 2000000 чаще допускают задолжности по кредиту чем клиенты с доходом от 200000 до 1000000.

### 3.4 Как разные цели кредита влияют на его возврат в срок?

Сгруппируем клинтов по столбцу (purpose_category), посчитаем общее количество клиентов в каждой группе, и клиентов имевших задолжности по кредитам. Данные выведем в сводной таблице data_purpose_category_pivot.

In [36]:
data_purpose_category_pivot = data.pivot_table(index=['purpose_category'], values=['debt'], aggfunc=('count', 'sum'))
data_purpose_category_pivot 

debt     
                          count  sum
purpose_category                    
операции с автомобилем     4279  400
операции с недвижимостью  10751  780
получение образования      3988  369
проведение свадьбы         2313  183

Высчитаем процентное отношение клентов с задолжностью к общему числу клиетнов. Данные выведем в колонке purpose_category_percent.

In [37]:
data_purpose_category_pivot['purpose_category_percent'] = data_purpose_category_pivot['debt','sum'] / data_purpose_category_pivot['debt','count'] * 100

data_purpose_category_pivot['purpose_category_percent'] = data_purpose_category_pivot['purpose_category_percent'].round(1)
data_purpose_category_pivot

debt      purpose_category_percent
                          count  sum                         
purpose_category                                             
операции с автомобилем     4279  400                      9.3
операции с недвижимостью  10751  780                      7.3
получение образования      3988  369                      9.3
проведение свадьбы         2313  183                      7.9

**Вывод:** Полученные данные говорят о том, что чаще всего не возвращают в срок кредиты цели которых "получение образования" или "операции с автомобилем".

### 3.5 Приведем возможные причины появления пропусков в исходных данных.

Одной из возможных причин появления пропусков в данных является умышленное сокрытие информации или ее не предоставление. Так же пропуски могут появляться если информация физически отсутсвуют.(Например у человека не может быть кредитной истории если он не разу не брал кредит, или отсутсвует трудовой стаж если человек не работал)

### 3.6 Объясните, почему заполнить пропуски медианным значением — лучшее решение для количественных переменных.

Если заполнять пропуски средним арифметическим, значение может получится некорректный результат, если в данных присутсвуют значения, который сильно отличаются от других. Медиана берет среднее значение из всего набора данных, а так же при расчете медианы можно использовать сгруппированные данные, чтоб медианное значение было еще достоверней.

##  4. Общий вывод

Исследование надежности заемщиков показало, что самыми ненадежными являются заемщики:
 - имеющие детей;
 - не состоящий в официальном браке; 
 - цель креда которых связана с автоболем или с получение образования.

Самыми надежными при этом будут заемщики у которых:

 - нет детей;
 - земщик состоял либо состоит в официальном браке;
 - цель кредита которых - это операции с недвижимостью.

Так как при разбивке клиентов на категории по уровню дохода в 3 из пяти групп количество клиентов оказалось много меньше, чем в двух других. По этому анализируя этот параметр нельзя дать однозначного ответа что уровень дохода влияет на возможность возникновения задолжности по кредиту. 